In [1]:
from tool import *

## OTD 실적 구하기

In [2]:
# 지난 주 OTD 실적 구하기
thisweek = get_weekname(datetime.date.today()) # 이번주 값 변수 저장
ref_week = get_weekname_from(thisweek, -1) # 지난 주 값 변수 저장
print(ref_week) # 지난주 이름 출력
with open('D:/Data/GSCP raw data.bin', 'rb') as f: # GSCP DB를 불러옴
    df1 = pickle.load(f)

# GSCP DB 에서 검색할 조건 셋팅
c1 = (df1['Ref'] == ref_week) # 지난주로 필터링할 조건
c2 = (df1['Ver'] == 'Latest') # version 명으로 필터링할 조건
c3 = (df1['From Site'] == 'Quanta') # 업체명으로 필터링할 조건
df1 = df1[c1 & c2 & c3] # 상기조건으로 DB 필터링하여 df1으로 저장
df1 = df1[df1['Updated_at'] == df1['Updated_at'].max()] #df1 에서 가장 최근의 update version만 한번더 필터링
df1 = df1[['Mapping Model.Suffix', 'To Site', ref_week]]
df1 = df1[df1[ref_week] > 0].reset_index(drop=True)
df1 = df1.rename(columns={ref_week:'Target'})
df1.insert(0, 'Ref', ref_week)
df1.tail(5)

2023-04-17(W16)


,Ref,Mapping Model.Suffix,To Site,Target
44,2023-04-17(W16),16T90R-K.ADB9U1,ENUS_V,193
45,2023-04-17(W16),16T90R-K.ADS9U1,ENUS_V,2
46,2023-04-17(W16),16TD90R-G.AX56K,EKHQ,2
47,2023-04-17(W16),16TD90R-G.AX76K,EKHQ,5
48,2023-04-17(W16),16TG90Q-G.AA75KN,EKHQ,80


In [3]:
df1['Target'].sum()

3511

In [4]:
with open('D:/Data/Quanta shipment result DB.bin', 'rb') as f:
    sr = pickle.load(f)

In [5]:
if sr[(sr['Week Name'] == ref_week) & (sr['RSD Week Name'] > ref_week)].shape[0] > 0: # 지난 주 선적 실적에 미래구간의 Pre ship이 있다면, 해당 Pre-Ship 값을 지난주 OTD target SP 리스트에 추가하는 로직
    print('There are pre-ship in {}'.format(ref_week))
    add_df = sr[(sr['Week Name'] == ref_week) & (sr['RSD Week Name'] > ref_week)][['Week Name', 'Mapping Model.Suffix', 'Ship To', 'Ship']].reset_index(drop=True)
    add_df.columns = df1.columns
    df1 = pd.concat([df1, add_df]).reset_index(drop=True).convert_dtypes()

There are pre-ship in 2023-04-17(W16)


In [6]:
df1['Target'].sum()

3769.0

In [7]:
sr = sr[sr['Week Name']==ref_week].pivot_table('Ship', index=['Week Name', 'Mapping Model.Suffix', 'Ship To'], aggfunc=sum).reset_index()
sr.columns = ['Ref', 'Mapping Model.Suffix', 'To Site', 'Ship']
sr

,Ref,Mapping Model.Suffix,To Site,Ship
0,2023-04-17(W16),11TG50Q-E.AC10KN,EKHQ,40.0
1,2023-04-17(W16),14T90Q-G.AP56ML,EKHQ,30.0
2,2023-04-17(W16),14T90Q-G.AP79L,EKHQ,16.0
3,2023-04-17(W16),14T90Q-G.AS50ML,EKHQ,10.0
4,2023-04-17(W16),14T90R-G.AA55A3,EASL,10.0
5,2023-04-17(W16),14T90R-G.AA56K,EKHQ,40.0
6,2023-04-17(W16),14T90R-G.AA5CK,EKHQ,85.0
7,2023-04-17(W16),14T90R-G.AA75A3,EASL,5.0
8,2023-04-17(W16),14T90R-G.AA75B,EEES,20.0
9,2023-04-17(W16),14T90R-G.AP7WL,EKHQ,36.0


In [8]:
df1 = df1.groupby(['Ref', 'Mapping Model.Suffix', 'To Site']).sum().reset_index()
df1.tail(5)

,Ref,Mapping Model.Suffix,To Site,Target
47,2023-04-17(W16),16T90R-K.ADB9U1,ENUS_V,193.0
48,2023-04-17(W16),16T90R-K.ADS9U1,ENUS_V,2.0
49,2023-04-17(W16),16TD90R-G.AX56K,EKHQ,2.0
50,2023-04-17(W16),16TD90R-G.AX76K,EKHQ,5.0
51,2023-04-17(W16),16TG90Q-G.AA75KN,EKHQ,80.0


In [9]:
otd_result = pd.merge(df1, sr, how='outer').fillna(0)
otd_result.loc[otd_result['Target'] < otd_result['Ship'], 'Target'] = otd_result.loc[otd_result['Target'] < otd_result['Ship'], 'Ship']
otd_result['Country'] = otd_result['To Site'].replace(site_map)
otd_result['OTD(Result)'] = round(otd_result['Ship'] / otd_result['Target'] * 100, 1)
otd_result = otd_result[['Ref', 'Country', 'Mapping Model.Suffix', 'To Site', 'Target', 'Ship', 'OTD(Result)']].convert_dtypes()
otd_result.insert(1, 'Series', otd_result['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model))
otd_result

,Ref,Series,Country,Mapping Model.Suffix,To Site,Target,Ship,OTD(Result)
0,2023-04-17(W16),11T50Q,Korea,11TG50Q-E.AC10KN,EKHQ,40.0,40,100.0
1,2023-04-17(W16),14T90Q,Korea,14T90Q-G.AP56ML,EKHQ,30.0,30,100.0
2,2023-04-17(W16),14T90Q,Korea,14T90Q-G.AP79L,EKHQ,30.0,16,53.3
3,2023-04-17(W16),14T90Q,Korea,14T90Q-G.AS50ML,EKHQ,10.0,10,100.0
4,2023-04-17(W16),14T90Q,US,14T90Q-K.AAB9U1,ENUS_V,50.0,0,0.0
5,2023-04-17(W16),14T90Q,US,14T90Q-K.APG5U1,ENUS_V,40.0,0,0.0
6,2023-04-17(W16),14T90R,Korea,14T90R-G.AA50K,EKHQ,63.0,0,0.0
7,2023-04-17(W16),14T90R,Singapore,14T90R-G.AA55A3,EASL,10.0,10,100.0
8,2023-04-17(W16),14T90R,Korea,14T90R-G.AA56K,EKHQ,40.0,40,100.0
9,2023-04-17(W16),14T90R,Korea,14T90R-G.AA5CK,EKHQ,90.0,85,94.4


In [10]:
otd_result['Target'].sum()

3931.0

In [11]:
otd_result['Ship'].sum()

2109

In [12]:
with open('D:/Data/OTD_result_db.bin', 'rb') as f:
    otd_db = pickle.load(f)

In [13]:
otd_db.shape

(945, 9)

In [14]:
otd_db = otd_db[~(otd_db['Ref'] == ref_week)]

In [15]:
otd_db.shape

(895, 9)

In [16]:
otd_db = pd.concat([otd_db, otd_result])

In [17]:
otd_db = otd_db.reset_index(drop=True)

In [18]:
otd_db.shape

(947, 9)

In [19]:
with open('D:/Data/OTD_result_db.bin', 'wb') as f:
    pickle.dump(otd_db, f)

In [20]:
with open('D:/Data/DB backup/OTD_result_db.bin', 'wb') as f:
    pickle.dump(otd_db, f)

### OTD 실적 보기

In [2]:
with open('D:/Data/OTD_result_db.bin', 'rb') as f:
    otd = pickle.load(f)

In [4]:
import plotly.express as px

In [5]:
otd.head(3)

,Ref,Series,Country,Mapping Model.Suffix,To Site,Target,Ship,OTD(Result),OTD Failure Reason
0,2022-12-19(W51),11T50Q,Korea,11TG50Q-E.AC10KN,EKHQ,630.0,576,91.4,NaN
1,2022-12-19(W51),14T90Q,Singapore,14T90Q-G.AA55A3,EASL,5.0,5,100.0,NaN
2,2022-12-19(W51),14T90Q,Korea,14T90Q-G.AA56K,EKHQ,80.0,0,0.0,NaN


In [8]:
# 주차별 OTD 실적
otd_week = otd.pivot_table(['Target', 'Ship'], index='Ref', aggfunc=sum)
otd_week['OTD(Result)'] = round(otd_week['Ship'] / otd_week['Target'] * 100, 1)
otd_week.reset_index()
px.bar(otd_week.reset_index(), x='Ref', y='OTD(Result)', width=500, height=400).update_xaxes(tickangle=70)

,Ref,Ship,Target,OTD(Result)
0,2022-12-19(W51),1891,6138.0,30.8
1,2022-12-26(W52),502,4509.0,11.1
2,2023-01-02(W01),3794,12079.0,31.4
3,2023-01-09(W02),765,9090.0,8.4
4,2023-01-16(W03),8535,19400.0,44.0
5,2023-01-23(W04),1352,12606.0,10.7
6,2023-01-30(W05),4669,10978.0,42.5
7,2023-02-06(W06),413,6779.0,6.1
8,2023-02-13(W07),2224,7886.0,28.2
9,2023-02-20(W08),7232,8626.0,83.8


In [13]:
# 모델별 OTD 실적
otd_model = otd.pivot_table(['Target', 'Ship'], index=['Ref', 'Series'], aggfunc=sum)
otd_model['OTD(Result)'] = round(otd_model['Ship'] / otd_model['Target'] * 100, 1)
otd_model

Ship  Target  OTD(Result)
Ref             Series                           
2022-12-19(W51) 11T50Q   576   630.0         91.4
                14T90Q    15   530.0          2.8
                15U40Q     0  1433.0          0.0
                15U50P  1009  1660.0         60.8
                15U50Q   291  1315.0         22.1
...                      ...     ...          ...
2023-04-10(W15) 15U50P    13    13.0        100.0
                15U50Q   222   340.0         65.3
                15U50R     0   100.0          0.0
                16T90Q    30   295.0         10.2
                16T90R   205   229.0         89.5

[138 rows x 3 columns]

array(['2023-04-10(W15)', '2023-04-03(W14)', '2023-03-27(W13)',
       '2023-03-20(W12)', '2023-03-13(W11)', '2023-03-06(W10)',
       '2023-02-27(W09)', '2023-02-20(W08)', '2023-02-13(W07)',
       '2023-02-06(W06)', '2023-01-30(W05)', '2023-01-23(W04)',
       '2023-01-16(W03)', '2023-01-09(W02)', '2023-01-02(W01)',
       '2022-12-26(W52)', '2022-12-19(W51)'], dtype=object)

In [18]:
# 국가별 OTD 실적
otd_country = otd.pivot_table(['Target', 'Ship'], index=['Ref', 'Country'], aggfunc=sum)
otd_country['OTD(Result)'] = round(otd_country['Ship'] / otd_country['Target'] * 100, 1)
otd_country

Ship   Target  OTD(Result)
Ref             Country                               
2022-12-19(W51) Germany        0     20.0          0.0
                Korea       1876   5648.0         33.2
                Poland         0     10.0          0.0
                Singapore     15     50.0         30.0
                Spain          0     40.0          0.0
                US             0    370.0          0.0
2022-12-26(W52) Austrailia     0     15.0          0.0
                Germany        5     80.0          6.2
                Korea        412   3949.0         10.4
                Poland         0     10.0          0.0
                Singapore      0     35.0          0.0
                Spain         20     40.0         50.0
                Taiwan         0     10.0          0.0
                US            65    370.0         17.6
2023-01-02(W01) Austrailia     0     15.0          0.0
                Canada         0     20.0          0.0
                Germany        0     75.0          0.0
                Korea       3794  11189.0         33.9
                Poland         0     10.0          0.0
                Singapore      0     35.0          0.0
                Spain          0     40.0          0.0
                Taiwan         0     10.0          0.0
                US             0    685.0          0.0
2023-01-09(W02) Austrailia    45     25.0        180.0
                Canada        40     40.0        100.0
                Germany        0    140.0          0.0
                Korea          0   7685.0          0.0
                Poland        10     10.0        100.0
                Singapore     35     35.0        100.0
                Spain         40     40.0        100.0
                Taiwan        10     10.0        100.0
                UK             5     15.0         33.3
                US           580   1090.0         53.2
2023-01-16(W03) Germany       90    205.0         43.9
                Korea       8115  18625.0         43.6
                UK            10     10.0        100.0
                US           320    560.0         57.1

### OTD 실적 수정

In [14]:
with open('D:/Data/OTD_result_db.bin', 'rb') as f:
    otd = pickle.load(f)

In [15]:
otd.shape

(676, 9)

In [16]:
otd = otd[~(otd['Ref'] == get_weekname_from(get_weekname(datetime.date.today()), -1))]

In [17]:
otd.shape

(653, 9)

In [18]:
with open('D:/Data/OTD_result_db.bin', 'wb') as f:
    pickle.dump(otd, f)

##  OTD 차질 사유 업데이트

In [2]:
with open('D:/Data/OTD_result_db.bin', 'rb') as f:
    otd = pickle.load(f)

In [3]:
week = '2023-02-13(W07)'

In [5]:
otd[otd['Ref'] == week]['Target'].sum()

7886.0

In [6]:
otd[otd['Ref'] == week]['Ship'].sum()

2224

In [17]:
reason_df = pd.read_clipboard()
reason_df = reason_df[['Ref', 'Mapping Model.Suffix', 'To Site', 'Target', 'Ship', 'OTD Failure Reason']]
reason_df

,Ref,Mapping Model.Suffix,To Site,Target,Ship,OTD Failure Reason
0,2023-02-13(W07),11TG50Q-E.AC10KN,EKHQ,780,0,Input 350pcs，camera 2/24*470
1,2023-02-13(W07),14T90P-G.ARTGK,EKHQ,49,0,2/17 issue PO
2,2023-02-13(W07),14T90Q-G.AA79K,EKHQ,50,0,2/21 Input
3,2023-02-13(W07),14T90Q-G.AP50ML,EKHQ,70,0,"shortage TOP,Cover"
4,2023-02-13(W07),14T90Q-G.AP56ML,EKHQ,100,50,2/19 Air
5,2023-02-13(W07),14T90Q-G.AP79L,EKHQ,60,0,"shortage TOP,Cover"
6,2023-02-13(W07),14T90Q-G.AR5GK,EKHQ,40,0,2/21 Input
7,2023-02-13(W07),14T90Q-G.AS50ML,EKHQ,40,0,"shortage TOP,Cover"
8,2023-02-13(W07),14T90Q-K.AA77A1,EEUK,10,0,2/21 Input
9,2023-02-13(W07),14T90Q-K.AAB9U1,ENUS_V,50,39,WIP


In [21]:
otd = otd.merge(reason_df, how='outer')

In [22]:
otd.head()

,Ref,Series,Country,Mapping Model.Suffix,To Site,Target,Ship,OTD(Result),OTD Failure Reason
0,2022-12-19(W51),11T50Q,Korea,11TG50Q-E.AC10KN,EKHQ,630.0,576,91.4,NaN
1,2022-12-19(W51),14T90Q,Singapore,14T90Q-G.AA55A3,EASL,5.0,5,100.0,NaN
2,2022-12-19(W51),14T90Q,Korea,14T90Q-G.AA56K,EKHQ,80.0,0,0.0,NaN
3,2022-12-19(W51),14T90Q,Singapore,14T90Q-G.AA75A3,EASL,25.0,10,40.0,NaN
4,2022-12-19(W51),14T90Q,Spain,14T90Q-G.AA78B,EEES,40.0,0,0.0,NaN


In [23]:
with open('D:/Data/OTD_result_db.bin', 'wb') as f:
    pickle.dump(otd, f)